In [1]:
import tensorflow as tf
import numpy as np
import glob
import cv2

SAVED_MODEL_DIR = "C:/Users/KK/Desktop/NPR2/exported_tflite_model/saved_model"
OUTPUT_TFLITE_MODEL = "C:/Users/KK/Desktop/NPR2/exported_tflite_model/detect_int8.tflite"

def representative_data_gen():
    image_paths = glob.glob("C:/Users/KK/Desktop/NPR2/images/train/*.jpg")
    for image_path in image_paths[:100]:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (320, 320))  # Match model input size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0  # ✅ Normalize to [0, 1]
        img = np.expand_dims(img, axis=0)
        yield [img]

model = tf.saved_model.load(SAVED_MODEL_DIR)
concrete_func = model.signatures["serving_default"]  # Extract function

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# 🔥 **Set Explicit Quantization Parameters**
converter.inference_input_type = tf.uint8  # Force INT8 input
converter.inference_output_type = tf.uint8  # Force INT8 output

# ✅ Convert Model
tflite_model = converter.convert()

# ✅ Save Model
with open(OUTPUT_TFLITE_MODEL, "wb") as f:
    f.write(tflite_model)

print("✅ MobileNetV2 SSD converted to FULL INT8!")

C:\Users\KK\yolov4-env\lib\site-packages\tensorflow\lite\python\convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


✅ MobileNetV2 SSD converted to FULL INT8!
